In [23]:
import yfinance as yf
from myUtils import yf_download_AdjOHLCV, pickle_dump, pickle_load, read_symbols_file  # NOQA
verbose = False  # True prints more output

# path_dir = "C:/Users/ping/MyDrive/stocks/MktCap2b_AUMtop1200/"
# path_data_dump = path_dir + "VSCode_dump/"
# path_symbols_file = path_dir + "source/"
# filename_symbols = path_symbols_file + '2021_Top1200_MktCap_n_AUM.txt'  # symbols text file
# filename_pickled_df_OHLCV_all_dates = 'df_OHLCV_all_dates'  # pickled filename
# filename_pickled_df_OHLCV_NYSE_dates = 'df_OHLCV_NYSE_dates'  # pickled filename
# filename_pickled_symbols_df_OHLCV_NYSE_dates =  'symbols_df_OHLCV_NYSE_dates'  # pickled filename

path_dir = "C:/Users/ping/Desktop/my_yfinance/"
path_data_dump = path_dir + "VSCode_dump/"
path_symbols_file = path_dir + "source/"
filename_symbols = path_symbols_file + "test_symbols_no_XOM.txt"  # symbols text file
filename_pickled_df_OHLCV_all_dates = "df_test_all_dates"  # pickled filename
filename_pickled_df_OHLCV_NYSE_dates = "df_test_NYSE_dates"  # pickled filename
filename_pickled_symbols_df_OHLCV_NYSE_dates = "symbols_df_test_NYSE_dates"  # pickled filename

In [24]:
mySymbols = read_symbols_file(filename_symbols)
print(mySymbols)

['AAPL', 'FTEC', 'TEST-SYM', 'BTC-USD', 'ETH-USD', 'test-sym2']


In [25]:
# download OHLCV, reindex to NSYE trading dates
index_symbol = "XOM"  # use Exxon's date as proxy for NYSE trading dates
df_XOM = yf.download(index_symbol)
# download OHLCV data for symbols in filename_symbols
df_OHLCV_all_dates, symbols_file = yf_download_AdjOHLCV(filename_symbols, verbose=verbose)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  6 of 6 completed

2 Failed downloads:
- TEST-SYM: No data found, symbol may be delisted
- TEST-SYM2: No data found, symbol may be delisted


In [44]:
df = df_OHLCV_all_dates.copy()
# get column names(i.e. symbols) from multi-index
#  column names are converted to all cap by yfinance download
symbols_df_OHLCV_all_dates = list(set(df.columns.get_level_values(0)))

In [45]:
symbols_all_NaN = []
for symbol in symbols_df_OHLCV_all_dates:
  # Check for symbols with all NaN dataframe
  # 1st .all returns a series of True/False, 2nd .all reduces to single True/False
  if df_OHLCV_all_dates[symbol].isna().all().all():
  # if df_OHLCV_NYSE_dates[symbol].isna().all().all():  
    print(f'Dataframe for {symbol} is all NaN')
    symbols_all_NaN.append(symbol)
print(f'symbols_all_NaN: {symbols_all_NaN}')

Dataframe for TEST-SYM is all NaN
Dataframe for TEST-SYM2 is all NaN
symbols_all_NaN: ['TEST-SYM', 'TEST-SYM2']


In [46]:
print(f'df_OHLCV_all_dates columns before dropping symbols with all NaN:\n{df_OHLCV_all_dates.columns.values.tolist()}\n')
for symbol in symbols_all_NaN:
  df_OHLCV_all_dates.drop(columns=[symbol], axis=1, level=0, inplace=True)
print(f'df_OHLCV_all_dates columns after dropping symbols with all NaN:\n{df_OHLCV_all_dates.columns.values.tolist()}')

df_OHLCV_all_dates columns before dropping symbols with all NaN:
[('TEST-SYM', 'Open'), ('TEST-SYM', 'High'), ('TEST-SYM', 'Low'), ('TEST-SYM', 'Close'), ('TEST-SYM', 'Adj Close'), ('TEST-SYM', 'Volume'), ('ETH-USD', 'Open'), ('ETH-USD', 'High'), ('ETH-USD', 'Low'), ('ETH-USD', 'Close'), ('ETH-USD', 'Volume'), ('FTEC', 'Open'), ('FTEC', 'High'), ('FTEC', 'Low'), ('FTEC', 'Close'), ('FTEC', 'Volume'), ('TEST-SYM2', 'Open'), ('TEST-SYM2', 'High'), ('TEST-SYM2', 'Low'), ('TEST-SYM2', 'Close'), ('TEST-SYM2', 'Adj Close'), ('TEST-SYM2', 'Volume'), ('BTC-USD', 'Open'), ('BTC-USD', 'High'), ('BTC-USD', 'Low'), ('BTC-USD', 'Close'), ('BTC-USD', 'Volume'), ('AAPL', 'Open'), ('AAPL', 'High'), ('AAPL', 'Low'), ('AAPL', 'Close'), ('AAPL', 'Volume')]

df_OHLCV_all_dates columns after dropping symbols with all NaN:
[('ETH-USD', 'Open'), ('ETH-USD', 'High'), ('ETH-USD', 'Low'), ('ETH-USD', 'Close'), ('ETH-USD', 'Volume'), ('FTEC', 'Open'), ('FTEC', 'High'), ('FTEC', 'Low'), ('FTEC', 'Close'), ('FTEC'

In [ ]:
# reindex to date index in df_XOM, weekend data are dropped
myNaN = float('nan')
df_OHLCV_NYSE_dates = df_OHLCV_all_dates.reindex(df_XOM.index, fill_value=myNaN)
df_OHLCV_NYSE_dates

In [ ]:
# https://stackoverflow.com/questions/25135578/python-pandas-drop-a-column-from-a-multi-level-column-index
# df = df_OHLCV_all_dates.copy()
df = df_OHLCV_NYSE_dates.copy()
# for sym in symbols_all_NaN:
#   df.drop(sym, axis=1, level=0, inplace=True)
my_list = df.columns.values.tolist()
my_list

In [ ]:
# pickle df_OHLCV and symbols
print(f"Full path to pickled df_OHLCV_all_dates:  {path_data_dump}{filename_pickled_df_OHLCV_all_dates}")
pickle_dump(df_OHLCV_all_dates, path_data_dump, filename_pickled_df_OHLCV_all_dates, verbose=verbose)
print(f"Full path to pickled df_OHLCV_NYSE_dates:  {path_data_dump}{filename_pickled_df_OHLCV_NYSE_dates}")
pickle_dump(df_OHLCV_NYSE_dates, path_data_dump, filename_pickled_df_OHLCV_NYSE_dates, verbose=verbose)
print(f"Full path to pickled symbols_df_OHLCV_NYSE_dates:  {path_data_dump}{filename_pickled_symbols_df_OHLCV_NYSE_dates}")
pickle_dump(symbols, path_data_dump, filename_pickled_symbols_df_OHLCV_NYSE_dates, verbose=verbose)

In [ ]:
TODO
break up symbols_file to small set
reload failed downloads
append fail downloads